In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)

In [4]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [5]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

In [6]:
import asyncio
import random
from typing import Any

from pydantic import BaseModel

from agents import Agent, RunContextWrapper, AgentHooks, Runner, Tool, function_tool

### Basic

In [7]:
class TestAgentHooks(AgentHooks):
    def __init__(self, ag_display_name):
        self.event_counter = 0
        self.ag_display_name = ag_display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} started. Usage: {context.usage}")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} ended. Usage: {context.usage}, Output: {output}")

In [10]:
starting_agent = Agent(
    name="Job Application Filter Agent",
    instructions=(
        "You are a job application filter agent. Your job is to review applications and flag any that "
        "appear incomplete, lack clarity, or require human review. We only accept applications that show "
        "clear experience in AI or software engineering."
    ),
    hooks=TestAgentHooks(ag_display_name="job_filter"),
    model=model
)

async def main():
    result = await Runner.run(
        starting_agent,
        input=(
            "<application>"
            "Hi, I'm Alex. I've worked in multiple fields and I'm interested in this position. "
            "Please let me know what the next steps are."
            "</application>"
        )
    )

    print(result.final_output)

asyncio.run(main())

### job_filter 1: Agent Job Application Filter Agent started. Usage: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
### job_filter 2: Agent Job Application Filter Agent ended. Usage: Usage(requests=1, input_tokens=81, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=145, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=226), Output: This application is flagged for the following reasons:

*   **Lack of Specificity:** The application doesn't mention the specific position being applied for.
*   **Vague Experience:** The applicant states they've "worked in multiple fields" without providing details about their experience or relevance to AI or software engineering.
*   **Unclear Qualifications:** There is no information about the applicant's skills or qualifications related to AI or software engineering

### Advance

In [9]:
class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended with output {output}"
        )

    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started tool {tool.name}"
        )

    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended tool {tool.name} with result {result}"
        )

In [12]:
@function_tool
def calculate_bmi(weight_kg: float, height_m: float) -> float:
    """Calculate BMI using weight and height."""
    return round(weight_kg / (height_m ** 2), 2)


@function_tool
def assess_risk(bmi: float) -> str:
    """Classify health risk based on BMI value."""
    if bmi < 18.5:
        return "Underweight – Consider gaining weight for optimal health."
    elif 18.5 <= bmi < 25:
        return "Normal – Keep maintaining your current lifestyle."
    elif 25 <= bmi < 30:
        return "Overweight – A balanced diet and regular exercise is recommended."
    else:
        return "Obese – High risk of health problems. Consult a medical professional."


class HealthAssessment(BaseModel):
    bmi: float
    risk: str


bmi_agent = Agent(
    name="BMI Evaluator Agent",
    instructions="Evaluate the BMI and return the corresponding health risk category.",
    tools=[assess_risk],
    hooks=CustomAgentHooks(display_name="BMI Evaluator"),
    model=model
)

starting_agent = Agent(
    name="Health Risk Assessor Agent",
    instructions="Calculate BMI based on input height and weight. Then hand off to BMI Evaluator Agent to assess health risk.",
    tools=[calculate_bmi],
    handoffs=[bmi_agent],
    hooks=CustomAgentHooks(display_name="Health Risk Assessor"),
    model=model
)


async def main() -> None:
    weight = float(input("Enter weight in kg: "))
    height = float(input("Enter height in meters: "))
    await Runner.run(
        starting_agent,
        input=f"My weight is {weight} kg and my height is {height} meters. Please assess my health risk.",
    )


asyncio.run(main())

Enter weight in kg: 68
Enter height in meters: 12
### (Health Risk Assessor) 1: Agent Health Risk Assessor Agent started
### (Health Risk Assessor) 2: Agent Health Risk Assessor Agent started tool calculate_bmi
### (Health Risk Assessor) 3: Agent Health Risk Assessor Agent ended tool calculate_bmi with result 0.47
### (Health Risk Assessor) 4: Agent Health Risk Assessor Agent handed off to BMI Evaluator Agent
### (BMI Evaluator) 1: Agent BMI Evaluator Agent started
### (BMI Evaluator) 2: Agent BMI Evaluator Agent started tool assess_risk
### (BMI Evaluator) 3: Agent BMI Evaluator Agent ended tool assess_risk with result Underweight – Consider gaining weight for optimal health.
### (BMI Evaluator) 4: Agent BMI Evaluator Agent ended with output Underweight – Consider gaining weight for optimal health.

